#  Automated ETL Pipeline for Banking Data Analysis



In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime


In [2]:
# URL and paths
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
exchange_rate_csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
csv_output_path = './Largest_banks_data.csv'
db_name = 'Banks.db'
table_name = 'Largest_banks'
log_file = 'code_log.txt'


In [3]:
# Setup logging
def log_progress(message):
    '''Logs the given message with a timestamp to the log file and prints it.'''
    timestamp_format = '%Y-%m-%d %H:%M:%S'  # Year-Month-Day-Hour-Minute-Second
    now = datetime.now()  # get current timestamp
    timestamp = now.strftime(timestamp_format)
    log_message = f"{timestamp} : {message}"
    with open(log_file, "a") as f:
        f.write(log_message + '\n')
    print(log_message)


In [4]:
# Make the first log entry
log_progress('Preliminaries complete. Initiating ETL process')

def extract(url):
    '''Extracts the table under the heading "By market capitalization" from the given URL to a Pandas dataframe.'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    
    headers = ['Name', 'MC_USD_Billion']
    df = pd.DataFrame(columns=headers)
    
    rows = table.find_all('tr')
    data = []
    for row in rows[1:11]:  # Top 10 banks
        cols = row.find_all('td')
        if len(cols) > 1:
            name = cols[1].text.strip()
            mc_usd = float(cols[2].text.strip().replace(',', '').replace('\n', ''))
            data.append({'Name': name, 'MC_USD_Billion': mc_usd})
    df = pd.DataFrame(data)
    
    log_progress("Data extraction complete. Initiating Transformation process")
    return df



2024-06-08 18:14:02 : Preliminaries complete. Initiating ETL process


In [5]:
#Transform
def transform(df, exchange_rate_csv_path):
    '''Transforms the dataframe by adding Market Capitalization in GBP, EUR, and INR based on exchange rates.'''
    exchange_rates = pd.read_csv(exchange_rate_csv_path, index_col=0).squeeze().to_dict()
    df['MC_GBP_Billion'] = [np.round(x * exchange_rates['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rates['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rates['INR'], 2) for x in df['MC_USD_Billion']]
    log_progress("Data transformation complete. Initiating Loading process")
    return df


In [6]:
#Load to csv
def load_to_csv(df, csv_path):
    '''Saves the dataframe to a CSV file.'''
    df.to_csv(csv_path, index=False)
    log_progress("Data saved to CSV file")



In [7]:
#Load to Database
def load_to_db(df, db_name, table_name):
    '''Saves the dataframe to a SQLite database.'''
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    log_progress("Data loaded to Database as a table, Executing queries")
    return conn


In [8]:
#Run Query
def run_query(query, conn):
    '''Runs a SQL query and prints the result.'''
    df = pd.read_sql(query, conn)
    print(df)
    log_progress("Process Complete")
    return df


In [9]:
# Run ETL process
df = extract(url)
df = transform(df, exchange_rate_csv_path)
load_to_csv(df, csv_output_path)

conn = load_to_db(df, db_name, table_name)

# Run queries on the database
query_1 = "SELECT * FROM Largest_banks"
query_2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
query_3 = "SELECT Name FROM Largest_banks LIMIT 5"

print("Query 1 Results:")
run_query(query_1, conn)

print("Query 2 Results:")
run_query(query_2, conn)

print("Query 3 Results:")
run_query(query_3, conn)

# Close SQLite3 connection
conn.close()
log_progress("Server Connection closed")

2024-06-08 18:14:03 : Data extraction complete. Initiating Transformation process
2024-06-08 18:14:03 : Data transformation complete. Initiating Loading process
2024-06-08 18:14:03 : Data saved to CSV file
2024-06-08 18:14:03 : Data loaded to Database as a table, Executing queries
Query 1 Results:
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   